# Reinforcement Learning Modell 

In [1]:
import socket
import json
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

# ===============================
#  MODEL DEFINITION
# ===============================

class TankController(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        return x

# Godot Connector

In [2]:
def start_server(host="127.0.0.1", port=5555):
    """Start a non-blocking socket server for Godot to connect to."""
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    s.bind((host, port))
    s.listen(1)
    s.settimeout(0.5)  # don't block forever on accept
    print(f"Python: listening on {host}:{port}")

    conn = None
    while conn is None:
        try:
            conn, addr = s.accept()
            print("Python: client connected", addr)
        except socket.timeout:
            # wait until Godot connects
            time.sleep(0.2)
            continue

    conn.settimeout(0.1)  # non-blocking receive
    return s, conn


# ===============================
#  MAIN LOOP (for Jupyter)
# ===============================

def run_godot_ai_loop(host="127.0.0.1", port=5555):
    """Main loop that talks to Godot and feeds a PyTorch model."""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = TankController().to(device)
    model.eval()

    s, conn = start_server(host, port)

    buf = ""
    print("Python: ready to receive data")

    try:
        while True:
            try:
                data = conn.recv(4096)
            except socket.timeout:
                # no data this tick
                time.sleep(0.01)
                continue

            if not data:
                print("Python: disconnected")
                break

            buf += data.decode()

            while "\n" in buf:
                line, buf = buf.split("\n", 1)
                line = line.strip()
                if not line:
                    continue

                try:
                    state = json.loads(line)
                except json.JSONDecodeError:
                    print("Bad JSON, skipping:", repr(line))
                    continue

                # ---- model inference ----
                obs = torch.tensor([
                    state["tank_x"],
                    state["tank_y"],
                    state["goal_x"],
                    state["goal_y"]
                ], dtype=torch.float32, device=device)

                with torch.no_grad():
                    action_tensor = model(obs)
                action = action_tensor.cpu().tolist()

                # ---- send action back to Godot ----
                conn.sendall((json.dumps(action) + "\n").encode())

            time.sleep(0.01)

    except KeyboardInterrupt:
        print("Python: manually stopped")

    finally:
        conn.close()
        s.close()
        print("Python: connection closed")



In [3]:
run_godot_ai_loop()

Python: listening on 127.0.0.1:5555
Python: client connected ('127.0.0.1', 51640)
Python: ready to receive data
Received state: {'goal_x': 0.761979166666667, 'goal_y': 0.425925925925926, 'tank_x': 0.281770833333333, 'tank_y': 0.421296296296296}
Sending: [0.06460744142532349, 0.08694470673799515]
Received state: {'goal_x': 0.761979166666667, 'goal_y': 0.425925925925926, 'tank_x': 0.281770833333333, 'tank_y': 0.421296296296296}
Sending: [0.06460744142532349, 0.08694470673799515]
Received state: {'goal_x': 0.761979166666667, 'goal_y': 0.425925925925926, 'tank_x': 0.281770833333333, 'tank_y': 0.421296296296296}
Sending: [0.06460744142532349, 0.08694470673799515]
Received state: {'goal_x': 0.761979166666667, 'goal_y': 0.425925925925926, 'tank_x': 0.281770833333333, 'tank_y': 0.421296296296296}
Sending: [0.06460744142532349, 0.08694470673799515]
Received state: {'goal_x': 0.761979166666667, 'goal_y': 0.425925925925926, 'tank_x': 0.281770833333333, 'tank_y': 0.421296296296296}
Sending: [0.064

ConnectionResetError: [Errno 104] Connection reset by peer